In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tqdm
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import pandas as pd
import cv2
import shutil
import glob

In [ ]:
def rebuild_dir(target_path):
    if os.path.exists(target_path):
        shutil.rmtree(target_path)
        os.makedirs(target_path)
    else:
        os.makedirs(target_path)

In [ ]:
workspace_path = os.getcwd() 
dataset_path=os.path.join(workspace_path,'Dataset')
source_path=os.path.join(dataset_path,'label_source')
color_range_dict = {'slide-2021-08-20T14-39-34-R1-S3_Wholeslide_Default_Extended.tif':[100,120,170,210,130,170],'slide-2021-11-02T21-16-24-R1-S5_Wholeslide_Default_Extended.tif':[100,120,170,210,150,190],'slide-2021-11-02T20-24-15-R1-S1_Wholeslide_Default_Extended.tif':[100,120,170,210,140,190]}

In [ ]:
pd.set_option('display.max_colwidth', 5500)

tif_file = [x for x in os.listdir(source_path) if x.split('.')[-1] == 'tif']
tif_dir = [os.path.join(source_path,x) for x in tif_file]

cropped_dir = [os.path.join(source_path,x.split('.')[0]+'_Cropped') for x in tif_file]
autolabeling_dir = [os.path.join(source_path,x.split('.')[0]+'_autolabeling') for x in tif_file]
df = pd.DataFrame({'file_name':tif_file,'file_dir':tif_dir,'cropped_dir':cropped_dir,'autolabeling_dir':autolabeling_dir,})
df

In [ ]:
testCase_1 = [246,247,253,279,280,340,341,365,392,401,402,418,419,445,446,456,479,484,490,523,524,552,553,584,585,617,708,725]
testCase_2 = [96,110,142,223,224]
testCase_3 = [34,65,81,84,91,97,100,110,111,117,125,139,150,169,170,186,187,188,193,219,223,224,248,249,279,305,306,351,352,369,396,424,429,452,460,469,486,487,512,536,544,550,577,624,674,720,730,731,732,733,734,768]

In [ ]:
#plt.figure(figsize=(10,10))
for idx in tqdm(df.index):
    try:
        print(idx)
        plt.imshow(cv2.imread(df.loc[idx,'file_dir'],cv2.IMREAD_COLOR))
        plt.show()
    except:
        print(idx,'ERROR')

plt.show()

In [ ]:
# CROP
for idx in tqdm(df.index):
    print(df.loc[idx,'file_name'])
    save_path = df.loc[idx,'cropped_dir']
    rebuild_dir(save_path)
    
    img_path = df.loc[idx,'file_dir']
    img_rgb=cv2.imread(img_path, cv2.IMREAD_COLOR)
    img_crop= []
    max_height,max_width = img_rgb.shape[0],img_rgb.shape[1]
    num=0
    for height in tqdm(range(0, img_rgb.shape[0], 512)):
        for width in range(0, img_rgb.shape[1], 512):
            
            img_rgb_crop = img_rgb[height:height+512, width:width+512, :]  # 30*30 절삭(높이, 너비)
            if height+512 > max_height or width+512 > max_width:
                pass
            else:
                if np.any(img_rgb_crop<210):
                    cv2.imwrite(os.path.join(save_path,'%05d.png' % num),img_rgb_crop)
                    num += 1
                    #cv2.imwrite(save_target+'/%05d.jpg' % num, img_rgb_crop)

In [ ]:
def Labeling(img_list,low,high,save_flag=False,save_target='',process_target=''):
    # 색상 범위 설정
    lower=low
    higher=high
    cnt=0
    if save_flag:
        os.makedirs(save_target,exist_ok=True)
    for img_name in tqdm(img_list):    
    #for img_name in tqdm(img_list, desc="Test ",leave=True) # 142:158 or 251:275 or 522:534
        
        img=cv2.imread(img_name)


        # 블러처리
        imgcopy=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        imgBlur = cv2.medianBlur(img, 55)   
        imgBlur2 = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2RGB)

        img_hsv = cv2.cvtColor(imgBlur2, cv2.COLOR_BGR2HSV)
        img_mask = cv2.inRange(img_hsv, lower, higher)
        img_result = cv2.bitwise_and(img, img, mask=img_mask)
        
        img_gray = cv2.cvtColor(img_result, cv2.COLOR_BGR2GRAY)
        ret, thr = cv2.threshold(img_gray, 20, 255, cv2.THRESH_BINARY) 
        contours,_ = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        thr_copy=np.copy(thr)
        cn_filled=cv2.drawContours(thr_copy, contours, -1, (255,255,255), -1)
        
        test = cv2.cvtColor(cn_filled,cv2.COLOR_GRAY2RGB)
        if save_flag:
            #print(os.path.join(save_target,'%05d.png' % cnt))
            cv2.imwrite(os.path.join(save_target,'%05d.png' % cnt), test)
            
        else :
            rebuild_dir(process_target)
            print(img_name)
            titles = ['original','imgBlur','HSV','Sementic', 'Threshold','contours']
            images = [imgcopy,imgBlur2,img_hsv ,img_result, thr,cn_filled]
            plt.figure(cnt,figsize=(15, 4),facecolor='white')
            for i in range(len(titles)):
                plt.subplot(1, 6, i+1)
                plt.imshow(images[i], cmap='gray')
                plt.title(titles[i])
                plt.axis('off')
                plt.tight_layout()
            plt.show()
        cnt+=1

In [ ]:
# VISUALIZE TEST 1
h1,h2,s1,s2,v1,v2 = color_range_dict[df.loc[0,'file_name']]
print(s1,s2,v1,v2)
img_list = sorted(glob.glob(df.loc[0,'cropped_dir']+'/*.png'))
test_case = [img_list[x] for x in testCase_1]
#Labeling(img_list[389:391],(h1,s1,v1-50),(h2,s2,v2),False)
Labeling(test_case,(h1,s1-10,v1-10),(h2,s2+10,v2+10),False,'',df.loc[0,'process_dir'])

In [ ]:
# VISUALIZE TEST 2
h1,h2,s1,s2,v1,v2 = color_range_dict[df.loc[1,'file_name']]
print(s1,s2,v1,v2)
img_list = sorted(glob.glob(df.loc[1,'cropped_dir']+'/*.png'))
test_case = [img_list[x] for x in testCase_2]
#Labeling(img_list[389:391],(h1,s1,v1-50),(h2,s2,v2),False)
Labeling(test_case,(h1,s1-10,v1-10),(h2,s2+10,v2+10),False,'',df.loc[1,'process_dir'])

In [ ]:
# VISUALIZE TEST 3
h1,h2,s1,s2,v1,v2 = color_range_dict[df.loc[2,'file_name']]
print(s1,s2,v1,v2)
img_list = sorted(glob.glob(df.loc[2,'cropped_dir']+'/*.png'))
test_case = [img_list[x] for x in testCase_3]
#Labeling(img_list[389:391],(h1,s1,v1-50),(h2,s2,v2),False)
Labeling(test_case,(h1,s1-10,v1-10),(h2,s2+10,v2+10),False,'',df.loc[2,'process_dir'])

In [ ]:
for idx in df.index:
    print(df.loc[idx,'file_name'])
    img_list= sorted(glob.glob(df.loc[idx,'cropped_dir']+'/*.png'))
    h1,h2,s1,s2,v1,v2 = color_range_dict[df.loc[idx,'file_name']]
    auto_labeling_path = df.loc[idx,'autolabeling_dir']
    rebuild_dir(auto_labeling_path)
    Labeling(img_list,(h1,s1-10,v1-10),(h2,s2+10,v2+10),True,auto_labeling_path)

In [ ]:
# Check
cropped = sorted(glob.glob(os.path.join(df.loc[0,'cropped_dir'],'*.png')))
test_cropped = [cropped[x] for x in testCase_1]
autolabel = sorted(glob.glob(os.path.join(df.loc[0,'autolabeling_dir'],'*.png')))
test_autolabel = [autolabel[x] for x in testCase_1]

for c,a in zip(test_cropped,test_autolabel):
    #print(c.split('/')[-1],a.split('/')[-1])
    plt.subplot(1,2,1)
    plt.title(c.split('/')[-1]+"_original")
    plt.axis('off')
    plt.imshow(cv2.imread(c))
    plt.subplot(1,2,2)
    plt.title(a.split('/')[-1]+"_autolabeling")
    plt.axis('off')
    plt.imshow(cv2.imread(a))
    plt.show()